In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------
# params (a adapter)
# -------------------
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "DOGEUSDT"]
DATA_DIR = "binance_public_data"
START = "2021-01-01"
END   = "2025-12-31"

WIN_VOL = 20
MA_TREND = 200

COST_BPS = 10
FREQ = 365


# -------------------
# load (csv Binance)
# -------------------
def load_symbol(sym):
    fn = f"{DATA_DIR}/{sym}_1d_2021_2025.csv"
    df = pd.read_csv(fn, parse_dates=["timestamp"], index_col="timestamp").sort_index()
    df = df.loc[START:END].copy()
    df["ret"] = np.log(df["close"] / df["close"].shift(1))
    return df.dropna(subset=["ret"])


# -------------------
# backtest
# -------------------
def run_bt(weights, rets, cost_bps=COST_BPS):
    # decalage poids (evite look-ahead)
    w = weights.shift(1).reindex(rets.index).fillna(0.0)

    # long-only + normalisation
    w = w.clip(lower=0.0)
    w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

    # perf brute
    gross = (w * rets).sum(axis=1)

    # turnover + couts
    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)

    # perf net
    net = gross - cost
    return net, to, w


def max_drawdown(eq):
    peak = eq.cummax()
    dd = eq / peak - 1.0
    return float(dd.min())


def stats(r, freq=FREQ):
    r = r.dropna()
    eq = (1 + r).cumprod()

    ann = float(eq.iloc[-1] ** (freq / len(r)) - 1) if len(r) > 0 else np.nan
    vol = float(r.std() * np.sqrt(freq))
    sharpe = float((r.mean() * freq) / (r.std() * np.sqrt(freq))) if r.std() > 0 else np.nan
    dd = max_drawdown(eq)

    return {"ann_return": ann, "ann_vol": vol, "sharpe": sharpe, "max_dd": dd}


# -------------------
# 1) build panel
# -------------------
dfs = {s: load_symbol(s) for s in SYMBOLS}

idx = None
for s in SYMBOLS:
    idx = dfs[s].index if idx is None else idx.intersection(dfs[s].index)

rets = pd.DataFrame({s: dfs[s].loc[idx, "ret"] for s in SYMBOLS}).dropna()
close = pd.DataFrame({s: dfs[s].loc[rets.index, "close"] for s in SYMBOLS})

# vol rolling
vol = rets.rolling(WIN_VOL).std()

# -------------------
# 2) Strategie conseillee:
# low-vol cross-section + trend gate BTC
# -------------------

# low-vol: on garde les actifs sous la mediane de vol chaque jour
q = vol.quantile(0.5, axis=1)
w_low = vol.le(q, axis=0).astype(float)

# trend gate BTC: si BTC sous sa MA200 -> cash (0)
btc = close["BTCUSDT"]
btc_ma = btc.rolling(MA_TREND).mean()
trend_on = (btc > btc_ma).astype(float).fillna(0.0)

w_strat = w_low.mul(trend_on, axis=0)

# normalisation "propre" avant bt (pas obligatoire mais plus clean)
w_strat = w_strat.div(w_strat.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# -------------------
# 3) baselines
# -------------------
w_eq = pd.DataFrame(1.0, index=rets.index, columns=rets.columns)

w_inv = 1.0 / vol.replace(0, np.nan)
w_inv = w_inv.fillna(0.0)
w_inv = w_inv.div(w_inv.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# -------------------
# 4) run
# -------------------
bt = {}
bt["equal_weight"] = run_bt(w_eq, rets, cost_bps=0)
bt["inv_vol"]      = run_bt(w_inv, rets, cost_bps=COST_BPS)
bt["lowvol_trend"] = run_bt(w_strat, rets, cost_bps=COST_BPS)

# stats
rows = []
for name, (r, to, _) in bt.items():
    st = stats(r)
    st["turnover_mean"] = float(to.mean())
    st["turnover_med"]  = float(to.median())
    st["name"] = name
    rows.append(st)

summary = pd.DataFrame(rows).set_index("name").sort_values("sharpe", ascending=False)
print(summary)

# equity plot
plt.figure(figsize=(12,5))
for name, (r, _, _) in bt.items():
    eq = (1 + r.fillna(0)).cumprod()
    plt.plot(eq.index, eq, label=name)

plt.legend()
plt.title("Equity curves (net)")
plt.tight_layout()
plt.show()

print("trend_on ratio:", float(trend_on.mean()))


![Image test 20](image/image_test_20.png)
